In [ ]:
# Carga de datos
import nbimporter  #  type: ignore
import functions  #  type: ignore
df_orig = functions.load_data()
df_orig.head()

In [3]:
# Componentes de tendencia lineal
df_orig = functions.add_linear_trend_component(df_orig)
df_orig.head(10)
def add_linear_trend_component(df):
    """Add linear trend component to the dataframe."""
    df = df.assign(trend=list(range(len(df))))
    return df

In [ ]:
# Componentes estacionales
df_orig = functions.add_month_component(df_orig)
df_orig.head(15)  

In [5]:
# Particionamiento de los datos
(
    X_complete,
    y_complete,
    X_train,
    y_train,
    X_test,
    y_test,
) = functions.train_test_split(
    df=df_orig,
    x_columns=["trend", "month"],
    y_column="yt_true",
)

In [6]:
# Especificación del Modelo
from sklearn.pipeline import Pipeline  #  type: ignore
from sklearn.preprocessing import (  #  type: ignore
    PolynomialFeatures,
    OneHotEncoder,
    MinMaxScaler,
)
from sklearn.linear_model import LinearRegression  #  type: ignore
from sklearn.compose import ColumnTransformer  #  type: ignore
def create_pipeline(degree):
    pipeline = Pipeline(
        [
            (
                "preprocessor",
                ColumnTransformer(
                    [
                        (
                            "trend",
                            Pipeline(
                                [
                                    ("poly", PolynomialFeatures(degree=degree)),
                                    ("scaler", MinMaxScaler()),
                                ]
                            ),
                            ["trend"],
                        ),
                        ("seasonal", OneHotEncoder(), ["month"]),
                    ]
                ),
            ),
            ("regressor", LinearRegression()),
        ]
    )

In [ ]:
# Entrenamiento y pronostico
for degree in range(1, 4):
    pipeline = create_pipeline(degree=degree)
    pipeline.fit(X_train, y_train)
    df_orig[f"yt_pred_time_trend_{degree}_dummy"] = pipeline.predict(X_complete)
functions.plot_time_series(df_orig)

In [ ]:
# Almacenamiento de los resultados
functions.save_forecasts(df_orig)  

In [ ]:
# Métricas de error
metrics = functions.compute_evaluation_metrics(df_orig)
functions.save_metrics(metrics)
metrics